In [4]:
import json
from langchain.llms import GooglePalm

with open("keys.json") as f:
    keys = json.load(f)

api_key = keys["api_key"]


llm = GooglePalm(google_api_key=api_key, temperature=0.2)
advice = llm("write love poem with name Dastan and Ilnura and write that ilnura broke dastan's heart")
print(advice)



Dastan and Ilnura,
Two hearts entwined,
Met one fateful night,
And their love was kindled.

They were inseparable,
Two halves of a whole,
And they vowed to be together,
Forever and a day.

But then, Ilnura's heart changed,
And she left Dastan heartbroken.
He could not understand,
Why she would do this to him.

He had loved her so deeply,
And he had given her his all.
But she had thrown it all away,
And he was left to pick up the pieces.

Dastan's heart was broken,
But he would not let it destroy him.
He would pick himself up,
And he would find love again.

He would not let Ilnura's betrayal,
Define his life.
He would move on,
And he would find happiness.

Dastan's heart was broken,
But he would not let it break him.


In [5]:
from langchain.utilities import SQLDatabase

# Extract the database credentials
db_user = keys['db_user']
db_password = keys['db_password']
db_host = keys['db_host']
db_name = keys['db_name']

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [6]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many white t-shirts do we have left for nike in extra small size and white color?")





> Entering new SQLDatabaseChain chain...
How many white t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(40,)]
Answer:40
> Finished chain.


In [7]:
qns1


{'query': 'How many white t-shirts do we have left for nike in extra small size and white color?',
 'result': '40'}